<div style='float: right'><img src='pic/sumline.png'/></div>
## <div id='sumline' />サムライン

In [ ]:
import numpy as np
from itertools import product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
....*.
.*....
...*.*
*.*...
....*.
.*....""".split('\n')
hh = [1687, 8361, 465, 721, 9139, 5700]
hw = [266, 597, 824, 829, 570, 352]
nw, nh = len(hw), len(hh)

### 問題
* 1から9の数字をいれます
* タテヨコ各列のカギは、その列に入る数の合計です
* タテもヨコも、同じ数字は１つまでです

### 変数
* v (1)
* r (2)

### 制約
* $v_{ij}$は1つ (3)
* rをvで表現 (4)
* 縦も横も同じ数字は1つまで (5)
* ヒントを満たすこと (6)

In [ ]:
def addsum(data, r, i, j, y, x):
    e, f = LpAffineExpression(), LpAffineExpression()
    while i < nh and j < nw:
        if data[i][j] == '.':
            f = 10*f + r[i,j]
        else:
            e, f = e+f, LpAffineExpression()
        i, j = i+y, j+x
    return e + f

In [ ]:
m = LpProblem()
v = np.array(addbinvars(nh, nw, 9)) # (1)
r = np.array(addvars(nh, nw)) # (2)
for i, j in product(range(nh), range(nw)):
    m += lpSum(v[i,j]) == 1 # (3)
    m += lpDot(range(9), v[i,j])+1 == r[i,j] # (4)
for k in range(9):
    for i in range(nh):
        m += lpSum(v[i,:,k]) <= 1 # (5)
    for j in range(nw):
        m += lpSum(v[:,j,k]) <= 1 # (5)
for i in range(nh):
    m += addsum(data, r, i, 0, 0, 1) == hh[i] # (6)
for j in range(nw):
    m += addsum(data, r, 0, j, 1, 0) == hw[j] # (6)
%time m.solve()
s = np.vectorize(value)(r).astype(int)
s[np.array([list(s) for s in data]) != '.'] = 0
print('\n'.join(''.join(str(c) if c else ' ' for c in t) for t in s))